# Trabalho 2

### Importando as bibliotecas que serão usadas no script inteiro

In [1]:
import math
import pandas  as pd
import numpy   as np

import matplotlib.pyplot as plt

from sklearn.model_selection  import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics          import mean_squared_error
from sklearn.preprocessing    import StandardScaler, MinMaxScaler, PolynomialFeatures, LabelBinarizer

from sklearn.linear_model     import LinearRegression, Ridge, Lasso, SGDRegressor
from sklearn.neighbors        import KNeighborsRegressor
from sklearn.ensemble         import RandomForestRegressor

from scipy.stats              import pearsonr

### Inserindo os dados do conjunto de treinamento e teste em variáveis

In [2]:
dados = pd.read_csv("conjunto_de_treinamento.csv")
dados_teste = pd.read_csv("conjunto_de_teste.csv")

Verificando as variáveis e a quantidade de cada uma para fazer o pré-processamento dos dados

In [3]:
for v in dados:
    print("\n%15s: " %v, "%4d categorias" % len(dados[v].unique() ) )
    print(dados[v].unique(), "\n")

# variaveis string: tipo (4), bairro (66), tipo_vendedor (2), diferenciais (83)


             Id:  4683 categorias
[2000 2001 2002 ... 6680 6681 6682] 


           tipo:     4 categorias
['Casa' 'Apartamento' 'Quitinete' 'Loft'] 


         bairro:    66 categorias
['Imbiribeira' 'Casa Amarela' 'Encruzilhada' 'Boa Viagem' 'Rosarinho'
 'Boa Vista' 'Espinheiro' 'Tamarineira' 'Gracas' 'Madalena' 'Parnamirim'
 'S Jose' 'Setubal' 'Arruda' 'Pina' 'Beira Rio' 'Caxanga' 'Casa Forte'
 'Prado' 'Iputinga' 'Campo Grande' 'Dois Irmaos' 'Torreao'
 'Ilha do Retiro' 'Areias' 'Varzea' 'Cordeiro' 'Torre' 'Santana' 'Barro'
 'Poco da Panela' 'Ipsep' 'Aflitos' 'Poco' 'Apipucos' 'Derby'
 'Cid Universitaria' 'Bongi' 'Jaqueira' 'Sto Amaro' 'Tejipio' 'Recife'
 'Monteiro' 'Macaxeira' 'Sancho' 'Afogados' 'San Martin' 'Cajueiro'
 'Hipodromo' 'Guabiraba' 'Engenho do Meio' 'Piedade' 'Jd S Paulo'
 'Lagoa do Araca' 'Ponto de Parada' 'Ilha do Leite' 'Estancia' 'Paissandu'
 'Zumbi' 'Agua Fria' 'Benfica' 'Soledade' 'Centro' 'Sto Antonio' 'Coelhos'
 'Cohab'] 


  tipo_vendedor:     2 categorias
['I

Verificar a quantidade de amostras de cada classe

In [4]:
print(dados["tipo_vendedor"].value_counts())
print(dados["tipo"].value_counts())
print(dados["bairro"].value_counts())
print(dados["diferenciais"].value_counts())

tipo_vendedor
Imobiliaria      4556
Pessoa Fisica     127
Name: count, dtype: int64
tipo
Apartamento    4501
Casa            177
Loft              3
Quitinete         2
Name: count, dtype: int64
bairro
Boa Viagem         1599
Madalena            327
Torre               240
Casa Amarela        225
Espinheiro          208
                   ... 
Ponto de Parada       1
Centro                1
Sto Antonio           1
Coelhos               1
Cohab                 1
Name: count, Length: 66, dtype: int64
diferenciais
piscina e churrasqueira                    1340
nenhum                                      986
piscina e sauna                             350
piscina e playground                        338
salao de festas                             187
                                           ... 
children care                                 1
sauna e quadra poliesportiva                  1
piscina e quadra de squash                    1
esquina e quadra poliesportiva                1
cam

Separando os bairros em 3 classes distintas para não haver tantos parâmetros para confundir o modelo. Além disso, tem muitos barros com apenas uma casa.

In [5]:
bairros = {  'Aflitos': 'classe_baixa',
             'Afogados':'classe_baixa',
             'Agua Fria':'classe_baixa',
             'Apipucos':'classe_media',
             'Areias':'classe_media',
             'Arruda':'classe_media',
             'Barro':'classe_baixa',
             'Beberibe':'classe_baixa',
             'Beira Rio':'classe_baixa',
             'Benfica':'classe_baixa',
             'Boa Viagem':'classe_alta',
             'Boa Vista':'classe_media',
             'Bongi':'classe_media',
             'Cajueiro':'classe_media',
             'Campo Grande':'classe_media',
             'Casa Amarela':'classe_media',
             'Casa Forte':'classe_alta',
             'Caxanga':'classe_media',
             'Centro':'classe_media',
             'Cid Universitaria':'classe_media',
             'Coelhos':'classe_baixa',
             'Cohab':'classe_baixa',
             'Cordeiro':'classe_media',
             'Derby':'classe_media',
             'Dois Irmaos':'classe_media',
             'Encruzilhada':'classe_media',
             'Engenho do Meio':'classe_media',
             'Espinheiro':'classe_alta',
             'Estancia':'classe_baixa',
             'Fundao':'classe_baixa',
             'Gracas':'classe_alta',
             'Guabiraba':'classe_baixa',
             'Hipodromo':'classe_baixa',
             'Ilha do Leite':'classe_media',
             'Ilha do Retiro':'classe_alta',
             'Imbiribeira':'classe_media',
             'Ipsep':'classe_media',
             'Ibura': 'classe_baixa',
             'Iputinga':'classe_media',
             'Jaqueira':'classe_alta',
             'Jd S Paulo':'classe_media',
             'Lagoa do Araca':'classe_media',
             'Macaxeira':'classe_baixa',
             'Madalena':'classe_alta',
             'Monteiro':'classe_media',
             'Paissandu':'classe_alta',
             'Parnamirim':'classe_media',
             'Piedade':'classe_baixa',
             'Pina':'classe_media',
             'Poco':'classe_baixa',
             'Poco da Panela':'classe_media',
             'Ponto de Parada':'classe_baixa',
             'Prado':'classe_media',
             'Recife':'classe_media',
             'Rosarinho':'classe_media',
             'S Jose':'classe_alta',
             'San Martin':'classe_media',
             'Sancho':'classe_media',
             'Santana':'classe_media',
             'Setubal':'classe_alta',
             'Soledade':'classe_media',
             'Sto Amaro':'classe_media',
             'Sto Antonio':'classe_media',
             'Tamarineira':'classe_alta',
             'Tejipio':'classe_media',
             'Torre':'classe_media',
             'Torreao':'classe_media',
             'Varzea':'classe_media',
             'Zumbi':'classe_baixa'
 }

Fazer o one hot encoding e a binarização das variáveis. Ademais, estou retirando a coluna de diferenciais porque ela tem muitos valores distintos e pode confundir o modelo.

In [6]:
dados = dados.drop(columns="diferenciais")
dados = dados.replace(bairros)

dados_teste = dados_teste.drop(columns="diferenciais")
dados_teste = dados_teste.replace(bairros)

dados = pd.get_dummies(dados, columns = ["tipo",
                                         "bairro",
                                         #"diferenciais"
                                        ])

dados_teste = pd.get_dummies(dados_teste, columns = ["tipo",
                                                     "bairro",
                                                     #"diferenciais"
                                                    ])

binarizador = LabelBinarizer()

for v in ["tipo_vendedor"]:
    dados[v] = binarizador.fit_transform(dados[v])
    dados_teste[v] = binarizador.fit_transform(dados_teste[v])

Verificar a correlação de pearson entre os parâmetros e o alvo (obs: os parâmetros não parecem muito promissores, mas veremos mais na hora de treinar o modelo):

In [7]:
pearson = dados.columns
for p in pearson:
  print(p, pearsonr(dados[p],dados['preco'])[0], pearsonr(dados[p],dados['preco'])[1])

Id -0.009642760756471807 0.5094355152393747
tipo_vendedor 0.12404728376340508 1.602213293941475e-17
quartos 0.020891610901120412 0.15287826107079835
suites 0.042130923648528294 0.0039311672431164985
vagas 0.029045578480214673 0.04686162008668862
area_util 0.02755707374185922 0.05934177388280898
area_extra 0.0011409571229646191 0.9377821777424706
churrasqueira 0.0158778131407028 0.2773303474775181
estacionamento -0.006366696356950981 0.6631438609162197
piscina 0.006808599466643847 0.6413514005318313
playground -0.012199105773287379 0.40393008010921216
quadra -0.004385607310944087 0.7641472772880071
s_festas 0.007544913686948061 0.6057242074163695
s_jogos -0.0025392812929621194 0.8620830607589952
s_ginastica -0.002295169564616567 0.8752272334126338
sauna 0.0011677995933596546 0.9363214991801263
vista_mar 0.0076576085351784725 0.6003503976654001
preco 0.9999999999999997 0.0
tipo_Apartamento -0.008536769452282827 0.5591888774445708
tipo_Casa 0.00900223160404917 0.5379652923350523
tipo_Loft

Retirando todos os valores que estão atrapalhando o modelo por estarem muito longe do esperado

In [8]:
dados = dados[(dados['preco']>= 50000) & (dados['preco']<= 5000000)]

Separação entre treino e teste

In [9]:
id = dados.loc[:, dados.columns == "Id"].to_numpy()
dados = dados.drop(columns="Id")
dados = dados.drop(columns="tipo_Quitinete")

id_teste = dados_teste.loc[:, dados_teste.columns == "Id"].to_numpy()
dados_teste = dados_teste.drop(columns="Id")

dados_embaralhados = dados.sample(frac=1, random_state=0)
x = dados_embaralhados.loc[:, dados.columns != "preco"].to_numpy()
y = dados_embaralhados.loc[:, dados.columns == "preco"].to_numpy()

x_treino, x_teste, y_treino, y_teste = train_test_split(
    x,
    y,
    train_size = 0.80
)

y_treino = y_treino.ravel()

escala = MinMaxScaler()
escala.fit(x_treino)

x_treino = escala.transform(x_treino)
x_teste  = escala.transform(x_teste)

### Random Forest

In [10]:
modelo = RandomForestRegressor(bootstrap=True, max_depth=2, min_samples_split=6,\
                               min_samples_leaf=2, n_estimators=210,random_state=5)

modelo = modelo.fit(x_treino,y_treino)

y_resposta_treino = modelo.predict(x_treino)
y_resposta_teste  = modelo.predict(x_teste)

mse  = mean_squared_error(y_treino, y_resposta_treino)
rmse = math.sqrt(mse)

print("%7s   %17.4f" % ("rmse" , rmse ))

x_arq_teste = dados_teste.loc[:, dados_teste.columns != "preco"].to_numpy()

y_pred_arq_teste = modelo.predict(x_arq_teste)

   rmse         325294.2194


Inserindo as respostas num arquivo csv

In [11]:
# arquivo_respostas = pd.read_csv('conjunto_de_teste.csv')
# resposta = pd.DataFrame({'Id': id_teste.ravel(),
#                          'preco':np.squeeze(np.int16(y_pred_arq_teste))})
# resposta.to_csv("resposta.csv",index=False)

### Regressor Linear

In [12]:
print('')
print("REGRESSOR LINEAR:")
print('')

print("Métrica   DENTRO da amostra   FORA da amostra")
print("-------   -----------------   ---------------")

regressor_linear = LinearRegression()

regressor_linear = regressor_linear.fit(x_treino, y_treino)

y_pred = cross_val_predict(regressor_linear, x_treino, y_treino, cv=5)

# y_resposta_treino = regressor_linear.predict(x_treino)
# y_resposta_teste  = regressor_linear.predict(x_teste)

# mse_in  = mean_squared_error(y_treino, y_resposta_treino)
# rmse_in = math.sqrt(mse_in)
# r2_in   = r2_score(y_treino, y_resposta_treino)
# #rmspe_in = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

# mse_out  = mean_squared_error(y_teste, y_resposta_teste)
# rmse_out = math.sqrt(mse_out)
# r2_out   = r2_score(y_teste, y_resposta_teste)
# #rmspe_out = (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))) * 100

# print("%7s   %17.4f   %15.4f" % ("mse"  , mse_in  , mse_out))
# print("%7s   %17.4f   %15.4f" % ("rmse" , rmse_in , rmse_out))
# print("%7s   %17.4f   %15.4f" % ("r2"   , r2_in   , r2_out))

#plt.scatter(x = y_teste, y = y_resposta_teste)

# fig=plt.figure()
# ax=fig.add_axes([0,0,1,1])
# ax.scatter(y_teste, y_resposta_teste, color='b', alpha=0.5)
# ax.set_ylim(-1000000,10000000)
# ax.set_xlim(-1000000,10000000)
# plt.show()

print("RMSE = %10.4f" % math.sqrt(mean_squared_error(y_treino,y_pred)))


REGRESSOR LINEAR:

Métrica   DENTRO da amostra   FORA da amostra
-------   -----------------   ---------------
RMSE = 342671.8064


### Regressor KNN

In [13]:
print('')
print(" REGRESSOR KNN:")
print('')

print(" K   DENTRO da amostra   FORA da amostra")
print("---   ----------------   ---------------")

for k in range (1,50):
    
    regressor_knn   = KNeighborsRegressor(
        n_neighbors = k,
        weights     = "uniform"      # uniform ou distance
    )
    
    regressor_knn = regressor_knn.fit(x_treino, y_treino)

    y_resposta_treino = regressor_knn.predict(x_treino)
    y_resposta_teste  = regressor_knn.predict(x_teste)

    mse_in  = mean_squared_error(y_treino, y_resposta_treino)
    rmse_in = math.sqrt(mse_in)
    
    mse_out  = mean_squared_error(y_teste, y_resposta_teste)
    rmse_out = math.sqrt(mse_out)
    
    print(" %3d   %17.4f   %15.4f" % (k , rmse_in , rmse_out))


 REGRESSOR KNN:

 K   DENTRO da amostra   FORA da amostra
---   ----------------   ---------------
   1          66484.4847       398308.4445
   2         176468.8319       340722.8685
   3         220797.9153       320039.1973
   4         244731.0356       319118.7382
   5         257835.3783       318613.5865
   6         269741.8248       314080.1969
   7         277373.1856       320069.9927
   8         286375.6575       322672.4986
   9         291149.0142       324069.6374
  10         294983.4110       323170.0279
  11         297846.1705       324983.8515
  12         303382.1840       324551.7999
  13         305849.4395       326126.3542
  14         307726.3861       326494.8983
  15         310605.7367       328003.8062
  16         312788.0508       329476.2991
  17         315788.8558       330924.7395
  18         318301.4416       332836.6947
  19         320423.9052       334511.8115
  20         322012.0315       335376.7230
  21         324871.0475       337194.96

### Regressor Polinomial

É possível observar que o melhor valor para o degree é 1

In [14]:
print('')
print(" REGRESSOR POLINOMIAL:")
print('')

print(" K   DENTRO da amostra   FORA da amostra")
print("---   ----------------   ---------------")

for k in range (1,4):

    pf = PolynomialFeatures(degree = k)

    pf = pf.fit(x_treino)

    x_treino_poly = pf.transform(x_treino)
    x_teste_poly  = pf.transform(x_teste)
    
    regressor_linear = LinearRegression()

    regressor_linear = regressor_linear.fit(x_treino_poly, y_treino)

    y_resposta_treino = regressor_linear.predict(x_treino_poly)
    y_resposta_teste  = regressor_linear.predict(x_teste_poly)

    mse_in  = mean_squared_error(y_treino, y_resposta_treino)
    rmse_in = math.sqrt(mse_in)
    
    mse_out  = mean_squared_error(y_teste, y_resposta_teste)
    rmse_out = math.sqrt(mse_out)
    
    print(" %3d   %17.4f   %15.4f" % (k , rmse_in , rmse_out))


 REGRESSOR POLINOMIAL:

 K   DENTRO da amostra   FORA da amostra
---   ----------------   ---------------
   1         328607.9948       342931.4986
   2         249018.2827   43951755752703144.0000
   3         186079.4642   4561776578465699.0000


### Regressão Polinomial com Regularização Ridge

É possível observar que o melhor valor para o grau é 2

In [15]:
print('')
print(" REGRESSOR POLINOMIAL COM REGULARIZAÇÃO RIDGE (L2):")
print('')

print(" K   DENTRO da amostra   FORA da amostra")
print("---   ----------------   ---------------")

for k in range (1,5):

    pf = PolynomialFeatures(degree = k)

    pf = pf.fit(x_treino)

    x_treino_poly = pf.transform(x_treino)
    x_teste_poly  = pf.transform(x_teste)

    for j in range (1,7):

        a = pow(10,j)
        regressor_ridge = Ridge(alpha = j)   # mudar o alpha para melhorar o modelo
    
        regressor_ridge = regressor_ridge.fit(x_treino_poly, y_treino)
    
        y_resposta_treino = regressor_ridge.predict(x_treino_poly)
        y_resposta_teste  = regressor_ridge.predict(x_teste_poly)
    
        mse_in  = mean_squared_error(y_treino, y_resposta_treino)
        rmse_in = math.sqrt(mse_in)
        
        mse_out  = mean_squared_error(y_teste, y_resposta_teste)
        rmse_out = math.sqrt(mse_out)
    
        print(" %3d   %3d   %17.4f   %15.4f" % (k , j, rmse_in , rmse_out))

pf = PolynomialFeatures(degree = 4)

pf = pf.fit(x_treino)

x_treino_poly = pf.transform(x_treino)
x_teste_poly  = pf.transform(x_teste)

regressor_ridge = Ridge(alpha = 10.0)   # mudar o alpha para melhorar o modelo

regressor_ridge = regressor_ridge.fit(x_treino_poly, y_treino)

x_arq_teste = dados_teste.loc[:, dados_teste.columns != "preco"].to_numpy()

x_poly_arq_teste = pf.transform(x_arq_teste)

y_pred_arq_teste = regressor_ridge.predict(x_poly_arq_teste)


 REGRESSOR POLINOMIAL COM REGULARIZAÇÃO RIDGE (L2):

 K   DENTRO da amostra   FORA da amostra
---   ----------------   ---------------
   1     1         329119.0880       338969.1612
   1     2         330187.0648       336598.5503
   1     3         331480.2275       335186.1799
   1     4         332844.3894       334368.7937
   1     5         334204.8629       333931.0975
   1     6         335526.3719       333741.5138
   2     1         277918.2624       302353.4792
   2     2         286364.0379       312327.0890
   2     3         291214.3107       316966.4694
   2     4         294616.2400       319603.8641
   2     5         297256.8250       321281.5677
   2     6         299433.3533       322432.1613


C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


   3     1         251671.5816       280471.6792


C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


   3     2         262166.9361       287596.6397


...

C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


   4     5         259113.3839       287167.9312


C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


   4     6         262378.1212       289496.2399


C:\Users\andre\anaconda3\Lib\site-packages\sklearn\utils\extmath.py:189: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


Inserindo as respostas num arquivo csv

In [16]:
arquivo_respostas = pd.read_csv('conjunto_de_teste.csv')
resposta = pd.DataFrame({'Id': id_teste.ravel(),
                         'preco':np.squeeze(np.int16(y_pred_arq_teste))})
resposta.to_csv("resposta.csv",index=False)

C:\Users\andre\AppData\Local\Temp\ipykernel_8424\3792132305.py:3: RuntimeWarning: invalid value encountered in cast
  'preco':np.squeeze(np.int16(y_pred_arq_teste))})


### Regressão Polinomial com Regularização Lasso

É possível observar que, após o grau aumente além de 2 ou 3, o modelo começa a ficar ruim. Logo, por motivos de memória, trunquei um pouco o range do k

In [17]:
print('')
print(" REGRESSOR POLINOMIAL COM REGULARIZAÇÃO LASSO (L1):")
print('')

print(" K   DENTRO da amostra   FORA da amostra")
print("---   ----------------   ---------------")

for k in range (1,3):

    pf = PolynomialFeatures(degree = k)

    pf = pf.fit(x_treino)

    x_treino_poly = pf.transform(x_treino)
    x_teste_poly  = pf.transform(x_teste)

    for j in range (1,10):
        a = pow(10,j)
        regressor_lasso = Lasso(alpha = j, max_iter = 10000)   # mudar o alpha para melhorar o modelo
    
        regressor_lasso = regressor_lasso.fit(x_treino_poly, y_treino)
    
        y_resposta_treino = regressor_lasso.predict(x_treino_poly)
        y_resposta_teste  = regressor_lasso.predict(x_teste_poly)
    
        mse_in  = mean_squared_error(y_treino, y_resposta_treino)
        rmse_in = math.sqrt(mse_in)
        
        mse_out  = mean_squared_error(y_teste, y_resposta_teste)
        rmse_out = math.sqrt(mse_out)
        
        print(" %3d   %3d   %17.4f   %15.4f" % (k , j, rmse_in , rmse_out))


 REGRESSOR POLINOMIAL COM REGULARIZAÇÃO LASSO (L1):

 K   DENTRO da amostra   FORA da amostra
---   ----------------   ---------------
   1     1         328608.0581       342929.6071
   1     2         328608.2478       342928.1442
   1     3         328608.5641       342927.1099
   1     4         328609.0068       342926.5042
   1     5         328609.5760       342926.3267
   1     6         328610.2716       342926.5779
   1     7         328611.0935       342927.2567
   1     8         328612.0421       342928.3650
   1     9         328613.1170       342929.9009


C:\Users\andre\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.243e+13, tolerance: 1.214e+11
  model = cd_fast.enet_coordinate_descent(


   2     1         248131.8560       372068.1487


C:\Users\andre\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.607e+13, tolerance: 1.214e+11
  model = cd_fast.enet_coordinate_descent(


   2     2         248411.6605       365636.2618


C:\Users\andre\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.671e+13, tolerance: 1.214e+11
  model = cd_fast.enet_coordinate_descent(


   2     3         248702.3465       359655.8274
   2     4         248945.0224       356525.1589
   2     5         249055.7661       353910.1715
   2     6         249129.8017       351315.1980
   2     7         249213.4933       348801.7639
   2     8         249309.6637       346342.9365
   2     9         249416.8968       344135.0246


### Regressor SGD

In [18]:
print('')
print(" REGRESSOR SGD:")
print('')

print("Métrica   DENTRO da amostra   FORA da amostra")
print("-------   -----------------   ---------------")

for k in range(1, 5):
    
    a = pow(10,k)
    
    regressor_sgd = SGDRegressor(loss = "squared_error",
                                 alpha = a,
                                 penalty = "l2",
                                 tol=1e-5,
                                 max_iter = 100000
                                )   # mudar hiperparâmetros para melhorar o modelo
    
    regressor_sgd = regressor_sgd.fit(x_treino, y_treino)
    
    y_resposta_treino = regressor_sgd.predict(x_treino)
    y_resposta_teste  = regressor_sgd.predict(x_teste)
    
    mse_in  = mean_squared_error(y_treino, y_resposta_treino)
    rmse_in = math.sqrt(mse_in)
    
    mse_out  = mean_squared_error(y_teste, y_resposta_teste)
    rmse_out = math.sqrt(mse_out)
    
    print("%7s   %17.4f   %15.4f" % ("rmse" , rmse_in , rmse_out))


 REGRESSOR SGD:

Métrica   DENTRO da amostra   FORA da amostra
-------   -----------------   ---------------


NameError: name 'r2_score' is not defined